# Day 32: Model Poisoning Detection

In this lab, we will simulate a **Data Poisoning Attack** and use outlier detection to defend against it.
We will generate a synthetic dataset and inject anomalous "poison" points.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.security.poisoning import PoisoningDetector

## 1. Generate Clean Data

We create a simple dataset with two clusters (e.g., normal training data).

In [ ]:
np.random.seed(42)

# Cluster 1
X1 = 0.3 * np.random.randn(100, 2) + [2, 2]
# Cluster 2
X2 = 0.3 * np.random.randn(100, 2) + [-2, -2]

X_clean = np.vstack((X1, X2))

## 2. Inject Poison

We add a few points far away from the main clusters. These represent poisoned samples (e.g., mislabeled or malicious inputs).

In [ ]:
# Poison points
X_poison = np.array([
    [0, 0],     # Point in the middle (could be bridge attack)
    [4, 4],     # Far outlier
    [-4, 4]     # Far outlier
])

X_combined = np.vstack((X_clean, X_poison))
y_true = np.array([0] * len(X_clean) + [1] * len(X_poison)) # 0=Clean, 1=Poison

## 3. Detect Poison

We train our `PoisoningDetector` on the combined data.

In [ ]:
detector = PoisoningDetector(contamination=0.02) # Expect low % of poison
detector.fit(X_combined)
preds = detector.detect(X_combined)

print(f"Detected {sum(preds)} potential poison samples.")

# Check accuracy against ground truth (simplistic check)
detected_poison_indices = np.where(preds)[0]
print(f"Detected indices: {detected_poison_indices}")
print(f"Correctly detected indices: {np.where(y_true == 1)[0]}")